In [1]:
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
import math

C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14228\1966561588.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_ecopetrol = pd.read_csv("ecopetrol.csv")
df_ecopetrol

,Date,Price,Open,High,Low,Vol.,Change %
0,03/01/2024,"2,240.0","2,280.0","2,290.0","2,200.0",9.33M,-2.18%
1,02/29/2024,"2,290.0","2,315.0","2,320.0","2,290.0",3.39M,-0.43%
2,02/28/2024,"2,300.0","2,360.0","2,380.0","2,300.0",4.52M,-2.54%
3,02/27/2024,"2,360.0","2,340.0","2,370.0","2,340.0",2.18M,0.64%
4,02/26/2024,"2,345.0","2,340.0","2,350.0","2,325.0",4.09M,0.64%
...,...,...,...,...,...,...,...
949,04/07/2020,"2,230.0","2,295.0","2,385.0","2,215.0",10.78M,-1.33%
950,04/06/2020,"2,260.0","2,270.0","2,315.0","2,180.0",10.53M,-0.44%
951,04/03/2020,"2,270.0","2,100.0","2,270.0","2,085.0",16.69M,9.13%
952,04/02/2020,"2,080.0","1,900.0","2,165.0","1,900.0",13.56M,13.04%


# Preprocesamiento

In [3]:
#Eliminar columnas inútiles
df_ecopetrol = df_ecopetrol.drop(columns=['Open','High','Low','Vol.','Change %'])

#Se invierte el dataframe para empezar desde los datos mas antiguos
df_ecopetrol = df_ecopetrol.iloc[::-1]
df_ecopetrol = df_ecopetrol.reset_index(drop=True)

#Convertir los precios de string a float
df_ecopetrol['Price'] = df_ecopetrol['Price'].str.replace(',', '')
df_ecopetrol['Price'] = df_ecopetrol['Price'].astype(float)

#Añade la columna Y_n definida en la guía
new_col = [0]
for i in range(1, len(df_ecopetrol)):
    new_col.append(float((df_ecopetrol['Price'][i]/df_ecopetrol['Price'][i-1]) - 1))
df_ecopetrol.insert(2, 'Yn', new_col, True)

#Eliminamos primer dato debido a que no se puede calcular Yn
df_ecopetrol = df_ecopetrol.drop([0])
df_ecopetrol = df_ecopetrol.reset_index(drop=True)

In [4]:
# Temporalmente ajusta las opciones de visualización para el DataFrame df_ecopetrol
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_ecopetrol)

           Date   Price         Yn
0    04/02/2020  2080.0  1.304e-01
1    04/03/2020  2270.0  9.135e-02
2    04/06/2020  2260.0 -4.405e-03
3    04/07/2020  2230.0 -1.327e-02
4    04/08/2020  2360.0  5.830e-02
5    04/13/2020  2250.0 -4.661e-02
6    04/14/2020  2225.0 -1.111e-02
7    04/15/2020  2095.0 -5.843e-02
8    04/16/2020  2090.0 -2.387e-03
9    04/17/2020  1950.0 -6.699e-02
10   04/20/2020  1905.0 -2.308e-02
11   04/21/2020  1840.0 -3.412e-02
12   04/22/2020  1920.0  4.348e-02
13   04/23/2020  1980.0  3.125e-02
14   04/24/2020  1910.0 -3.535e-02
15   04/27/2020  1950.0  2.094e-02
16   04/28/2020  2075.0  6.410e-02
17   04/29/2020  2070.0 -2.410e-03
18   04/30/2020  2090.0  9.662e-03
19   05/04/2020  1960.0 -6.220e-02
20   05/05/2020  1985.0  1.276e-02
21   05/06/2020  1950.0 -1.763e-02
22   05/07/2020  1975.0  1.282e-02
23   05/08/2020  2040.0  3.291e-02
24   05/11/2020  2065.0  1.225e-02
25   05/12/2020  1995.0 -3.390e-02
26   05/13/2020  1920.0 -3.759e-02
27   05/14/2020  193

# Definición de estados

In [5]:
#calcula estados basados en el valor de la columna 'Yn' en relación con la desviación estándar de una ventana de datos de tamaño l
new_col = []
l = 20
data = df_ecopetrol['Yn']

for i, percentage in enumerate(data):
    if i == len(data) - 1:
        min = i - l + 1
        desviacion = np.std(data[min:i])

    if percentage >= 0:
        new_col.append('sube')
    else:
        new_col.append('baja')

# Inserta la columna de estados calculados en la posición 2 del DataFrame df_bancolombia
df_ecopetrol.insert(2, 'Estado', new_col, True)
# Reestablece los índices del DataFrame df_bancolombia después de eliminar las filas
df_bancolombia = df_ecopetrol.reset_index(drop=True)

In [6]:
df_ecopetrol

,Date,Price,Estado,Yn
0,04/02/2020,2080.0,sube,0.130435
1,04/03/2020,2270.0,sube,0.091346
2,04/06/2020,2260.0,baja,-0.004405
3,04/07/2020,2230.0,baja,-0.013274
4,04/08/2020,2360.0,sube,0.058296
...,...,...,...,...
948,02/26/2024,2345.0,sube,0.006438
949,02/27/2024,2360.0,sube,0.006397
950,02/28/2024,2300.0,baja,-0.025424
951,02/29/2024,2290.0,baja,-0.004348


In [7]:
estados_posibles =['baja','sube']  # Posibles estados del sistema

total_cols = len(estados_posibles)  # Calcula el número total de columnas, que es igual al número de estados posibles
total_rows = total_cols * total_cols  # Calcula el número total de filas como el cuadrado del número de estados posibles


In [8]:
#matriz de transición para un modelo Markoviano
estados = np.array(df_ecopetrol['Estado'])

markovian_table = np.array(estados)
estados = np.delete(estados, 0)
estados = np.append(estados, 0)
markovian_table = np.vstack([markovian_table,estados])
estados = np.delete(estados, 0)
estados = np.append(estados, 0)
markovian_table = np.vstack([markovian_table,estados])

markovian_table = markovian_table.T
markovian_table = np.delete(markovian_table, [-1, -2], axis=0)
markovian_table 

array([['sube', 'sube', 'baja'],
       ['sube', 'baja', 'baja'],
       ['baja', 'baja', 'sube'],
       ...,
       ['sube', 'sube', 'baja'],
       ['sube', 'baja', 'baja'],
       ['baja', 'baja', 'baja']], dtype=object)

In [9]:
#frecuencias esperadas de transición entre estados
expected_freq = [[0] * (total_cols + 1) for i in range(total_cols)]

for i in range(len(markovian_table)):
    actual = markovian_table[i][1]
    row_index = estados_posibles.index(actual)
    col_index = estados_posibles.index(markovian_table[i][2])

    expected_freq[row_index][col_index] += 1
    expected_freq[row_index][total_cols] += 1

cols = estados_posibles.copy()
cols.append('Recuento')
expected_freq_df = pd.DataFrame(expected_freq, columns=cols)
cols.pop(-1)
expected_freq_df.insert(0, 'T_Actual', cols)
expected_freq_df

,T_Actual,baja,sube,Recuento
0,baja,223,236,459
1,sube,237,255,492


# Modelo - Matriz de transición

In [10]:
# probabilidad de transición entre los estados del modelo Markoviano
model_freq = [[0] * total_cols for i in range(total_cols)]
total_freq = [0,0,0,0,0,0,0,0]

for i in range(len(markovian_table)):
    actual = markovian_table[i][1]
    row_index = estados_posibles.index(actual)
    col_index = estados_posibles.index(markovian_table[i][2])

    model_freq[row_index][col_index] += 1
    total_freq[row_index] += 1

model_transition_prob_df = pd.DataFrame(model_freq, columns=estados_posibles)
model_transition_prob_df.insert(0, 'Actual', estados_posibles)

for i in range(total_cols):
    for j in range(1,total_cols+1):
        model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)

model_transition_prob_df

C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14228\868081643.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.485838779956427' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14228\868081643.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.514161220043573' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)


,Actual,baja,sube
0,baja,0.485839,0.514161
1,sube,0.481707,0.518293


In [12]:
print("La última desviación medida es de: " + str(desviacion))

La última desviación medida es de: 0.011025860992791493
